****Sentinel 2 Worflow****

This workflow loops over all rectangles of the csv and converts them to ee.Geomtries.
It will find all Sentinel 2 images in a given timefrime that have a cloud coverage less than X and stores their IDs in list.

A nested loop will run over all these images of the list and atmospherically corrects them to Sentinel 2A Level. All corrected images are stored in a new Image Collection upon which a Cloud Mask is applied. 
Lastly, the image is calculated with the median of the collection and then exported to the drive (Bands: B2, B3, B4, B8 and B11).
Alternatively, instead of the mean one could also apply a mosaic.
Atmospheric correction is performed based on Py6S (wilson, 2013) and Murphy (2018) Atmospheric script. Marked functions are adopted from Murphy (2018)


This jupyter-notebook was written and executed in Google Colab. All files are written to the authors personal drive.

Murphy, S. (2018), ‘Sam Murphy Github’.
URL: https://github.com/samsammurphy?tab=repositories

Wilson, R. T. (2013), ‘Py6s: A python interface to the 6s radiative transfer model.’,Computers & Geosciences51(2), 166

In [0]:
import ee
from ee import batch
from IPython.display import display, Image

# Install Py6s which is not pre installed on Colab
#Credits to Robin Wilson for this package
!pip install Py6S
!wget http://rtwilson.com/downloads/sixsV1.1 -O /usr/local/bin/sixsV1.1
!chmod a+x /usr/local/bin/sixsV1.1
!apt-get install libgfortran3
from Py6S import *

#import other packages
import datetime
import math
import os
import sys

from google.colab import drive
drive.mount('/gdrive/')
sys.path.append('/gdrive/My Drive/Colab Notebooks')
from atmosphere import Atmospheric # Murphy 2018
# Trigger the authentication flow.
ee.Authenticate()

ee.Initialize()

In [0]:
def spectralResponseFunction(bandname):
    """
    Extract spectral response function for given band name
    Function by Sam Murphy
    """

    bandSelect = {
        'B1':PredefinedWavelengths.S2A_MSI_01,
        'B2':PredefinedWavelengths.S2A_MSI_02,
        'B3':PredefinedWavelengths.S2A_MSI_03,
        'B4':PredefinedWavelengths.S2A_MSI_04,
        'B5':PredefinedWavelengths.S2A_MSI_05,
        'B6':PredefinedWavelengths.S2A_MSI_06,
        'B7':PredefinedWavelengths.S2A_MSI_07,
        'B8':PredefinedWavelengths.S2A_MSI_08,
        'B8A':PredefinedWavelengths.S2A_MSI_8A,
        'B9':PredefinedWavelengths.S2A_MSI_09,
        'B10':PredefinedWavelengths.S2A_MSI_10,
        'B11':PredefinedWavelengths.S2A_MSI_11,
        'B12':PredefinedWavelengths.S2A_MSI_12,
        }
    
    return Wavelength(bandSelect[bandname])

In [0]:
def toa_to_rad(bandname):
    """
    Converts top of atmosphere reflectance to at-sensor radiance
    Function by Sam Murphy
    """
    # solar exoatmospheric spectral irradiance
    ESUN = info['SOLAR_IRRADIANCE_'+bandname]
    solar_angle_correction = math.cos(math.radians(solar_z))
    
    # Earth-Sun distance (from day of year)
    doy = scene_date.timetuple().tm_yday
    d = 1 - 0.01672 * math.cos(0.9856 * (doy-4))# http://physics.stackexchange.com/questions/177949/earth-sun-distance-on-a-given-day-of-the-year
   
    # conversion factor
    multiplier = ESUN*solar_angle_correction/(math.pi*d**2)

    # at-sensor radiance
    rad = toa.select(bandname).multiply(multiplier)
    
    return rad

In [0]:
def surface_reflectance(bandname):
    """
    Calculate surface reflectance from at-sensor radiance given waveband name
    Function by Sam Murphy
    """

    # run 6S for this waveband
    s.wavelength = spectralResponseFunction(bandname)
    s.run()
    
    # extract 6S outputs
    Edir = s.outputs.direct_solar_irradiance             #direct solar irradiance
    Edif = s.outputs.diffuse_solar_irradiance            #diffuse solar irradiance
    Lp   = s.outputs.atmospheric_intrinsic_radiance      #path radiance
    absorb  = s.outputs.trans['global_gas'].upward       #absorption transmissivity
    scatter = s.outputs.trans['total_scattering'].upward #scattering transmissivity
    tau2 = absorb*scatter                                #total transmissivity
    
    # radiance to surface reflectance
    rad = toa_to_rad(bandname)
    ref = rad.subtract(Lp).multiply(math.pi).divide(tau2*(Edir+Edif))
    
    return ref

In [0]:
# Sam Murphy

def ESAclouds(toa):
    """
    European Space Agency (ESA) clouds from 'QA60', i.e. Quality Assessment band at 60m
    parsed by Nick Clinton
    Function by Sam Murphy
    """

    qa = toa.select('QA60')

    # bits 10 and 11 are clouds and cirrus
    cloudBitMask = int(2**10)
    cirrusBitMask = int(2**11)

    # both flags set to zero indicates clear conditions.
    clear = qa.bitwiseAnd(cloudBitMask).eq(0).And(\
           qa.bitwiseAnd(cirrusBitMask).eq(0))

    # cloud is not clear
    cloud = clear.eq(0)

    return cloud

def shadowMask(toa,cloudMask):
    """
    Finds cloud shadows in images
    Originally by Gennadii Donchyts, adapted by Ian Housman
    Function by Sam Murphy
    """

    def potentialShadow(cloudHeight):
        """
        Finds potential shadow areas from array of cloud heights
        returns an image stack (i.e. list of images) 
        """
        cloudHeight = ee.Number(cloudHeight)

        # shadow vector length
        shadowVector = zenith.tan().multiply(cloudHeight)

        # x and y components of shadow vector length
        x = azimuth.cos().multiply(shadowVector).divide(nominalScale).round()
        y = azimuth.sin().multiply(shadowVector).divide(nominalScale).round()

        # affine translation of clouds
        cloudShift = cloudMask.changeProj(cloudMask.projection(), cloudMask.projection().translate(x, y)) # could incorporate shadow stretch?

        return cloudShift

    # solar geometry (radians)
    azimuth = ee.Number(toa.get('solar_azimuth')).multiply(math.pi).divide(180.0).add(ee.Number(0.5).multiply(math.pi))
    zenith  = ee.Number(0.5).multiply(math.pi ).subtract(ee.Number(toa.get('solar_zenith')).multiply(math.pi).divide(180.0))

    # find potential shadow areas based on cloud and solar geometry
    nominalScale = cloudMask.projection().nominalScale()
    cloudHeights = ee.List.sequence(500,4000,500)        
    potentialShadowStack = cloudHeights.map(potentialShadow)
    potentialShadow = ee.ImageCollection.fromImages(potentialShadowStack).max()

    # shadows are not clouds
    potentialShadow = potentialShadow.And(cloudMask.Not())

    # (modified) dark pixel detection 
    darkPixels = toa.normalizedDifference(['green', 'swir1']).gt(0.25)

    # shadows are dark
    shadow = potentialShadow.And(darkPixels).rename(['shadows'])

    return shadow
  

ESAclouds = ESAclouds
shadowMask = shadowMask


def sentinel2mask(img):
  """
  Masks cloud (and shadow) pixels from Sentinel 2 image
  Function by Sam Murphy
  """

  # top of atmosphere reflectance
  toa = img.select(['B1','B2','B3','B4','B6','B8','B8A','B9','B10', 'B11', 'B12'],\
    ['aerosol', 'blue', 'green', 'red', 'red2','red3','red4','h2o', 'cirrus','swir1', 'swir2'])\
    .divide(10000).addBands(img.select('QA60'))\
    .set('solar_azimuth',img.get('MEAN_SOLAR_AZIMUTH_ANGLE'))\
    .set('solar_zenith',img.get('MEAN_SOLAR_ZENITH_ANGLE'))
                
  # ESA clouds
  ESAcloud = ESAclouds(toa)

  # Shadow
  shadow = shadowMask(toa, ESAcloud)

  # cloud and shadow mask
  mask = ESAcloud.Or(shadow).eq(0)

  return img.updateMask(mask).toFloat()

In [0]:
# sepcify a list of polygons that will be used as areas of interest. The wkt list are the 96 polygons of Uganda
wkt_liste = [[[29.510949100000005, -1.5210635559999999], [29.510949100000005, -1.034092893], [30.04218312, -1.034092893], [30.04218312, -1.5210635559999999], ...
print(wkt_liste)

In [0]:

'''
    This part is designed by Johanna Kauffert
    It loops over the list of coordinates and finds all images in the given time frame 
    with a certain percentage of cloudcover, atmospherically corrects the images
    and saves the mosaic to teh drive
    '''
globalcount = 0
for iwkt in wkt_liste:
    #transfer coordinates to GEE Coordinates
    geom= ee.Geometry.Polygon(iwkt)
    #Find images in the time frame, geometry and given cloud cover
    collection = (ee.ImageCollection("COPERNICUS/S2")
              # Filter for images within a given date range.
              .filter(ee.Filter.date('2015-06-28', '2016-12-28'))
              # Filter for images that overlap with the assigned geometry.
              .filterBounds(geom)
              # Filter for images that have less then 15% cloud coverage.
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
            )
    # make a dictionary with infos of the collection
    coll_dict = collection.getInfo()
    info = coll_dict["properties"]
    features = coll_dict["features"]
    listID = []
    #loop over all features in the dict
    for i in range(len(features)):
      #dont use more than 30 images
      if i < 30:
          #save the name of the image in a list
          p = features[i]['properties']['system:index']
          print(p)
          listID.append(p)
    print("Appeneded all to the list")
    
    output_list = [] #create new image collection
    # loop over all images of the list and apply atmospheric correction
    counter = 0
    for m in listID:
        S2 = ee.Image(f'COPERNICUS/S2/{m}')
        toa = S2.divide(10000)
        info = S2.getInfo()['properties']
        date = ee.Date(datetime.datetime.utcfromtimestamp(info['system:time_start']/1000))
        #print(date)


        ### Credits to SAM MURPHY ####
        scene_date = datetime.datetime.utcfromtimestamp(info['system:time_start']/1000)# i.e. Python uses seconds, EE uses milliseconds
        #print(scene_date)
        solar_z = info['MEAN_SOLAR_ZENITH_ANGLE']
        h2o = Atmospheric.water(geom,date).getInfo()
        o3 = Atmospheric.ozone(geom,date).getInfo()
        aot = Atmospheric.aerosol(geom,date).getInfo()
        SRTM = ee.Image('CGIAR/SRTM90_V4')# Shuttle Radar Topography mission covers *most* of the Earth
        alt = SRTM.reduceRegion(reducer = ee.Reducer.mean(),geometry = geom.centroid()).get('elevation').getInfo()
        km = alt/1000 # i.e. Py6S uses units of kilometers
        s = SixS()

        # Atmospheric constituents
        s.atmos_profile = AtmosProfile.UserWaterAndOzone(h2o,o3)
        s.aero_profile = AeroProfile.Continental
        s.aot550 = aot

        # Earth-Sun-satellite geometry
        s.geometry = Geometry.User()
        s.geometry.view_z = 0               # always NADIR (I think..)
        s.geometry.solar_z = solar_z        # solar zenith angle
        s.geometry.month = scene_date.month # month and day used for Earth-Sun distance
        s.geometry.day = scene_date.day     # month and day used for Earth-Sun distance
        s.altitudes.set_sensor_satellite_level()
        s.altitudes.set_target_custom_altitude(km)
        output = S2.select('QA60')
        for band in ['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B10','B11','B12']:
            output = output.addBands(surface_reflectance(band))
        # append now the corrected images to a new list
        output_list.append(output)
        counter += 1
        print(f'Processed Image: {counter}, Name: {m}')
    # make a new image collection from the list and the corrected images
    coll = ee.ImageCollection(output_list)

    #and have a first look at the first image of the collection
    channels = ['B4','B3','B2']
    first_look = ee.Image(
      ee.ImageCollection(output_list) 
          .first()
      )

    original = Image(url=first_look.select(channels).getThumbUrl({
                    'scale':100,
                    'min':0,
                    'max':0.25
                    }))
    display(original)
    my_dict = coll.getInfo()
    # apply a cloud mask to the image
    cloudfree = (ee.ImageCollection(output_list)
              .filterBounds(geom)
              #.select(['B4', 'B3', 'B2', 'QA60'])
              .map(sentinel2mask)
            )
    # only select the bands, that are important for the research
    cloudfree2 = (ee.ImageCollection(cloudfree)
                  #.sort('CLOUDY_PIXEL_PERCENTAGE', opt_ascending=False)
                  .select(['B4', 'B3', 'B2','B8','B11'])
                  )
    #calcualte the median
    cloudfree3 = cloudfree2.median()
    #Alternative way:
    #cloudfreeimage = cloudfree2.sort('system:index', opt_ascending=False).mosaic()
    
    ### Save the image to the folder 
    print(f"Geomtrie:  {globalcount}")
    
    task_config = {
        'region': geom,
        'folder': 'Sentinel2_1',
        'scale': 10,
        'crs': 'EPSG:4326',
        'description': f'UgandaS2_{globalcount}'
    }

    # Export Image
    task = ee.batch.Export.image.toDrive(cloudfree3, **task_config)
    task.start()
    task.status()
    globalcount += 1